#Same as last exercise but with DB.  We will use Chroma as a introductory learning DB, evenually will use postgresql (with Vector extension) for production.

We introduce here:
>> langchain retriever

Still not doing similarity search here

In [ ]:

from dotenv import load_dotenv
from pathlib import Path
import os
load_dotenv()
dotenv_path = Path('.env_secure')
load_dotenv(dotenv_path=dotenv_path)
open_api_key = os.environ.get('OPENAI_API_KEY')
dotenv_path = Path('.env_rag_0005')
load_dotenv(dotenv_path=dotenv_path)
max_filename_length = int(str(os.environ.get('MAX_FILENAME_LENGTH')))
url_strip_num_characters = int(os.environ.get('URL_STRIP_NUM_CHARACTERS'))
file_indicator_size = int(os.environ.get('FILE_INDICATOR_SIZE'))
text_chunk_size = int(os.environ.get('TEXT_CHUNK_SIZE'))
text_chunk_overlap = int(os.environ.get('TEXT_CHUNK_OVERLAP'))
output_directory = os.environ.get('OUTPUT_DIRECTORY')
file_selection = os.environ.get("FILE_SELECTION")
url_file_name = os.environ.get('URL_FILE_NAME') + '_' + file_selection + '.txt'
query_file_name = os.environ.get('QUERY_FILE_NAME') + '_' + file_selection + '.txt'
input_directory = os.environ.get('INPUT_DIRECTORY')
max_documents = int(os.environ.get('MAX_DOCUMENTS'))
chroma_directory = os.environ.get('CHROMA_DIRECTORY')
print(max_documents)
#logging_is_active_str = os.environ.get('LOGGING_IS_ACTIVE')
#logging_is_active = False
#if logging_is_active_str == 'True':
#    logging_is_active = True


In [ ]:

import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
logging.debug('test debug')
logging.info('test info')
logging.warning('test warning')
logging.error('test error')
logging.critical('test critical')

In [ ]:

from langchain.llms import OpenAI 
from bs4 import BeautifulSoup as bs
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from tqdm import tqdm
import tqdm

someCounter = 0
maxCounter = 0


In [ ]:
def get_file_contents(specified_file_name):
    file_full_name = './' + input_directory + '/' + specified_file_name
    text_file = open(file_full_name, "r")
    data = text_file.read()
    all_lines = data.splitlines()
    return all_lines

In [ ]:
def clear_files():
    cleared_count = 0
    dir = './' + output_directory
    for f in os.listdir(dir):
        os.remove(os.path.join(dir,f))
        cleared_count += 1
    return cleared_count


In [ ]:
def flush_files(base_text, base_name):
    global someCounter
    global maxCounter
    logging.info(f'')
    string_split = str(someCounter).zfill(file_indicator_size)
    filename = base_name + string_split
    try:
        f = open(f"./{output_directory}/{filename}.txt", "x")
        f.write(base_text)
        f.close()
        maxCounter += 1
    except:
        print(f'file exists {filename}, skipping')
        logging.info(f'file exists {filename}, skipping')
        return False
    return True

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

def check_f(file_to_be_checked):
    full_file_to_be_checked = './' + input_directory + '/' + file_to_be_checked
    check_file = os.path.isfile(full_file_to_be_checked)
    print(check_file)
    if (not(check_file)):
        print(f'error on file {full_file_to_be_checked}')
        raise StopExecution

check_f(url_file_name)
check_f(query_file_name)

In [ ]:
#this gets text from dynamic or static web page
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless=new")
def retrieve_text_from_url(inbound_url):
    driver = webdriver.Chrome(options=options)
    driver.get(inbound_url)
    html = driver.page_source
    soup = bs(html)
    print(soup)
    all_text = ''
    for tag in soup.find_all('p'):
        all_text += tag.text + '\n'
    return(all_text)


In [ ]:
ret = clear_files()
print(f"{ret} old files cleared")

In [ ]:
class StopExecution(Exception):
    def _render_traceback_(self):
        return []

def check_f(file_to_be_checked):
    full_file_to_be_checked = './' + input_directory + '/' + file_to_be_checked
    check_file = os.path.isfile(full_file_to_be_checked)
    print(check_file)
    if (not(check_file)):
        print(f'error on file {full_file_to_be_checked}')
        raise StopExecution

check_f(url_file_name)
check_f(query_file_name)


In [ ]:
url_list = get_file_contents(url_file_name)
print(url_list)
print(query_file_name)
query_list = get_file_contents(query_file_name)
print(query_list)



In [ ]:

url_alternate_names = []
for aitem in url_list:
    aitem_short = aitem[url_strip_num_characters:]
    aitem_short = aitem_short.replace("/", "_")
    aitem_short = aitem_short[:max_filename_length]
    url_alternate_names.append(aitem_short)


In [ ]:
# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



In [ ]:
#once we have a text chunk then we need to save it to a file
#In this example what we do is save each text chunk to a text file
#we will later then load the text chunks
import time

for someUrl in enumerate(url_list):
    global someCounter
    someCounter = 0 
    url_name = someUrl[1]
    url_index = someUrl[0]
    all_text_from_url = retrieve_text_from_url(url_name)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=text_chunk_size, chunk_overlap=text_chunk_overlap)
    someSplit = text_splitter.split_text(all_text_from_url)
    for one_text in someSplit:
        if(maxCounter >= max_documents):
            break
        ret_val = flush_files(one_text, url_alternate_names[url_index])
        someCounter += 1
    time.sleep(5)




In [ ]:
#from langchain.text_splitter import CharacterTextSplitter
#def get_text_chunks_langchain(text):
#   text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=100)
#   docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
#   return docs

In [ ]:
llm = OpenAI(temperature=0)

Here is where we start the questioning, first without the LLM knowing what we are doing.

In [ ]:
for specific_query in query_list:
    print(specific_query)

    print(llm(specific_query))

In [ ]:
from langchain.document_loaders import TextLoader

file_path = './' + output_directory
print(file_path)
# Load multiple files
loaders = [TextLoader(os.path.join(file_path, fn)) for fn in os.listdir(file_path)]
print(loaders)

all_documents = []
for loader in loaders:
    print("Loading raw document" + loader.file_path)
    raw_documents = loader.load()
    all_documents.extend(raw_documents)

print(all_documents)


In [ ]:
#now we will save the documents to chromadb
chroma_directory_full = './' + chroma_directory
db = Chroma.from_documents(documents=all_documents, embedding=embedding_function, persist_directory=chroma_directory_full)
db.persist()
db = None
print(chroma_directory_full)
db = Chroma(persist_directory=chroma_directory_full,
            embedding_function=embedding_function)
retriever = db.as_retriever()
print(all_documents)


#https://smngeo.medium.com/vector-search-with-large-language-models-ee1aaa4bcccf
#good easy example vectordb.similarity_search(query, k=5) 

#https://www.reddit.com/r/LangChain/comments/12qdgq7/whats_the_difference_between/

#https://www.timescale.com/blog/how-to-build-llm-applications-with-pgvector-vector-store-in-langchain/
#good example on similarity search and all usage of PGVector

This is from https://community.openai.com/t/do-retrieval-augmented-generation-systems-answer-out-of-context-questions/317460

I recommend running this script with and without this as context.

A question is never out-of-context or out of domain-answerable knowledge, unless you specifically instruct the AI that it is. The GPT-3+ models already comes with the ability to answer most anything you would feed it already.



In [ ]:
#https://github.com/samwit/langchain-tutorials/blob/main/YT_HF_Instructor_Embeddings_Chroma_DB_Multi_Doc_Retriever_LangChain_Part2.ipynb
#https://python.langchain.com/docs/use_cases/question_answering/how_to/vector_db_qa
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

for specific_query in query_list:
    print(specific_query)
    #also note can use chain.run
    query_response = chain(specific_query)
    print(query_response['query'])
    print('>>' + query_response['result'])
    #print(query_response.query)
    #print(query_response.result)

#response = chain(query)
#print(response)

#query = "Please summarize RAG and what it is useful for"
#response = chain(query)
#print(response)

#query = "Please explain some important concepts in RAG and why it is so important to the field of Artifical Intellegence"
#response = chain.run(query)
#print(response)



In [ ]:
#same exercise but using ChromaDB
